# MLFlow v2 protocol elasticnet wine example

In this example we are going to build a model using mlflow, pack and deploy it on seldon-core on a local kind cluster

Prerequisites:

- install and configure `mc`, follow the relevant section in this [link](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html):

- run this jupyter notebook in conda environment
```bash
$ conda create --name python3.8-mlflow-example python=3.8 -y
$ conda activate python3.8-mlflow-example
$ pip install jupyter
$ jupyter notebook
```

## Setup local kind cluster using ansible

### Install ansible prerequisites

In [18]:
!pip install ansible openshift docker passlib

  Using cached ansible-4.3.0-py3-none-any.whl
  Using cached openshift-0.12.1-py3-none-any.whl
  Using cached passlib-1.7.4-py2.py3-none-any.whl (525 kB)
  Using cached ansible_core-2.11.3-py3-none-any.whl
  Using cached python_string_utils-1.0.0-py3-none-any.whl (26 kB)
  Using cached ruamel.yaml-0.17.10-py3-none-any.whl (108 kB)
  Using cached kubernetes-12.0.1-py2.py3-none-any.whl (1.7 MB)
  Using cached resolvelib-0.5.4-py2.py3-none-any.whl (12 kB)
  Using cached cryptography-3.4.7-cp36-abi3-manylinux2014_x86_64.whl (3.2 MB)
  Using cached google_auth-1.34.0-py2.py3-none-any.whl (152 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.1.1-py2.py3-none-any.whl (146 kB)
  Using cached ruamel.yaml.clib-0.2.6-cp38-

In [19]:
!ansible-galaxy collection install git+https://github.com/SeldonIO/ansible-k8s-collection.git

Cloning into '/home/sa/.ansible/tmp/ansible-local-3444016e_43ifbj/tmpl593c8ki/ansible-k8s-collectiongefpw6fb'...
remote: Enumerating objects: 676, done.
remote: Counting objects: 100% (676/676), done.
remote: Compressing objects: 100% (417/417), done.
remote: Total 676 (delta 282), reused 536 (delta 166), pack-reused 0
Receiving objects: 100% (676/676), 277.25 KiB | 2.50 MiB/s, done.
Resolving deltas: 100% (282/282), done.
Your branch is up-to-date with 'origin/master'.
Starting galaxy collection install process
Process install dependency map
Starting collection install process
Installing 'seldonio.k8s:0.3.0-dev' to '/home/sa/.ansible/collections/ansible_collections/seldonio/k8s'
Created collection for seldonio.k8s:0.3.0-dev at /home/sa/.ansible/collections/ansible_collections/seldonio/k8s
seldonio.k8s:0.3.0-dev was installed successfully
Skipping 'kubernetes.core:2.1.1' as it is already installed
Skipping 'community.docker:1.9.0' as it is already installed
Skipping 'community.crypto:1

### Setup kind cluster and install seldon-core

In [31]:
%%bash
cd ../ansible
ansible-playbook playbooks/kind_cluster.yaml
ansible-playbook playbooks/main.yaml -e seldon_core_version=master 


PLAY [Create Kind Cluster and Install MetalLB] *********************************

TASK [seldonio.k8s.kind : Check if Kind v0.11.1 already downloaded] ************
ok: [localhost]

TASK [seldonio.k8s.kind : Create .resources directory if does not exist: /home/sa/code/seldon-core/ansible/.resources/] ***
ok: [localhost]

TASK [seldonio.k8s.kind : Download Kind v0.11.1 binary] ************************
skipping: [localhost]

TASK [seldonio.k8s.kind : Make Kind v0.11.1 binary executable] *****************
ok: [localhost]

TASK [seldonio.k8s.kind : Check if KinD Cluster already exists: ansible] *******
ok: [localhost]

TASK [seldonio.k8s.kind : Start KinD Cluster: 'ansible'] ***********************
skipping: [localhost]

TASK [seldonio.k8s.kind : Echo output of "kind create cluster ..." command] ****
skipping: [localhost]

TASK [seldonio.k8s.kind : Export kubeconfig for KinD Cluster: 'ansible'] *******
changed: [localhost]

TASK [seldonio.k8s.kind : Create a k8s namespace: seldon] *********

[WARNING]: The default idempotency check can fail to report changes in certain
cases. Install helm diff for better results.


## Train elasticnet wine model using `mlflow`

### Install `mlflow` and required dependencies to train the model

In [8]:
!pip install mlflow scikit-learn==0.23.2 pandas

  Using cached conda_pack-0.6.0-py2.py3-none-any.whl


### Define where the model artifacts will be saved

In [3]:
import os
from pathlib import Path
MODEL_DIR = Path(os.getcwd()) / "elasticnet_wine_model"

### Define training

In [4]:
# Wine Quality Sample a copy from: 
# https://github.com/mlflow/mlflow/blob/master/examples/sklearn_elasticnet_wine/train.ipynb

def train(in_alpha, in_l1_ratio):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.save_model(lr, MODEL_DIR)
        print(f" Model saved to {MODEL_DIR}")

### Train the elasticnet_wine model

In [6]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319587
  R2: 0.10862644997792614
 Model saved to /home/sa/code/seldon-core/notebooks/elasticnet_wine_model


### Install dependencies to be able to pack and deploy the model on `seldon_core`

We are going to use [`conda-pack`](https://conda.github.io/conda-pack/) to pack the python enviornment. We also need `mlserver` dependencies.
We are planning to simplify this workflow in future releases.

In [60]:
!pip install conda-pack mlserver==0.4.0 mlserver-mlflow==0.4.0

  Using cached mlserver-0.4.0-py3-none-any.whl (61 kB)
  Using cached mlserver_mlflow-0.4.0-py3-none-any.whl (8.3 kB)
  Using cached uvicorn-0.14.0-py3-none-any.whl (50 kB)
  Using cached orjson-3.6.1-cp38-cp38-manylinux_2_24_x86_64.whl (233 kB)
  Using cached fastapi-0.68.0-py3-none-any.whl (52 kB)
  Using cached grpcio-1.39.0-cp38-cp38-manylinux2014_x86_64.whl (4.3 MB)
  Using cached starlette-0.14.2-py3-none-any.whl (60 kB)
  Using cached pydantic-1.8.2-cp38-cp38-manylinux2014_x86_64.whl (13.7 MB)
  Using cached typing_extensions-3.10.0.0-py3-none-any.whl (26 kB)


  Using cached asgiref-3.4.1-py3-none-any.whl (25 kB)
  Using cached h11-0.12.0-py3-none-any.whl (54 kB)


### Pack the conda enviornment

In [61]:
import conda_pack
env_file_path = MODEL_DIR / "environment.tar.gz"
conda_pack.pack(
    output=str(env_file_path),
    force=True,
    verbose=True,
    ignore_editable_packages=False,
    ignore_missing_files=True,
)

Packing environment at '/home/sa/miniconda3/envs/python3.8-mlflow-example' to '/home/sa/code/seldon-core/notebooks/elasticnet_wine_model/environment.tar.gz'
[########################################] | 100% Completed | 42.9s


'/home/sa/code/seldon-core/notebooks/elasticnet_wine_model/environment.tar.gz'

### Configure `mc` to access the minio service in the local kind cluster
note: make sure that minio ip is reflected properly below, run `kubectl get service -n minio-system`

In [ ]:
!mc config host add minio-seldon http://172.18.255.3:9000 minioadmin minioadmin

### Copy the model artifacts to minio

In [75]:
import os
os.system("mc mb minio-seldon/model")
os.system(f"mc cp --recursive {MODEL_DIR} minio-seldon/model")

Removing `minio-seldon/model/MLmodel`.
Removing `minio-seldon/model/conda.yaml`.
Removing `minio-seldon/model/environment.tar.gz`.
Removing `minio-seldon/model/model.pkl`.
Removing `minio-seldon/model/requirements.txt`.
`/home/sa/code/seldon-core/notebooks/elasticnet_wine_model/environment.tar.gz` -> `minio-seldon/model/elasticnet_wine_model/environment.tar.gz`
`/home/sa/code/seldon-core/notebooks/elasticnet_wine_model/model.pkl` -> `minio-seldon/model/elasticnet_wine_model/model.pkl`
`/home/sa/code/seldon-core/notebooks/elasticnet_wine_model/requirements.txt` -> `minio-seldon/model/elasticnet_wine_model/requirements.txt`
`/home/sa/code/seldon-core/notebooks/elasticnet_wine_model/conda.yaml` -> `minio-seldon/model/elasticnet_wine_model/conda.yaml`
`/home/sa/code/seldon-core/notebooks/elasticnet_wine_model/MLmodel` -> `minio-seldon/model/elasticnet_wine_model/MLmodel`


mc: <ERROR> Unable to make bucket `minio-seldon/model`. Your previous request to create the named bucket succeeded and you already own it.


Total: 0 B, Transferred: 322.90 MiB, Speed: 469.47 MiB/s


0

In [63]:
%%writefile ./resources/mlflow_elasticnet_wine_v2.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  protocol: kfserving  # Activate v2 protocol
  name: wines
  predictors:
    - graph:
        children: []
        implementation: MLFLOW_SERVER
        modelUri: s3:/model
        envSecretRefName: seldon-rclone-secret
        name: classifier
      name: default
      replicas: 1

Overwriting ./resources/elasticnet_wine_v2.yaml


In [66]:
!kubectl apply -f ./resources/mlflow_elasticnet_wine_v2.yaml

seldondeployment.machinelearning.seldon.io/mlflow created


In [68]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=mlflow -o jsonpath='{.items[0].metadata.name}')

deployment "mlflow-default-0-classifier" successfully rolled out


In [73]:
import json

import requests

inference_request = {
    "parameters": {
        "content_type": "pd"
    },
    "inputs": [
        {
          "name": "fixed acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [7.4],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "volatile acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.7000],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "citric acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [0],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "residual sugar",
          "shape": [1],
          "datatype": "FP32",
          "data": [1.9],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "chlorides",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.076],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "free sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [11],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "total sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [34],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "density",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.9978],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "pH",
          "shape": [1],
          "datatype": "FP32",
          "data": [3.51],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "sulphates",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.56],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "alcohol",
          "shape": [1],
          "datatype": "FP32",
          "data": [9.4],
          "parameters": {
              "content_type": "np"
          }
        },
    ]
}

# note is the local balancer for istion, make sure that the ip is reflected in the setup,
# run kubectl get service -n istio-system
endpoint = "http://172.18.255.1/seldon/seldon/mlflow/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "classifier",
  "model_version": "v1",
  "id": "1b0648b3-b7ad-426f-9d37-395ab441b179",
  "parameters": null,
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "FP64",
      "parameters": null,
      "data": [
        5.576883936610762
      ]
    }
  ]
}


In [65]:
!kubectl delete -f ./resources/mlflow_elasticnet_wine_v2.yaml

seldondeployment.machinelearning.seldon.io "mlflow" deleted
